# Stable Diffusion Web UI Colab

A browser interface based on Gradio library for Stable Diffusion running on Google Colab.
For more detailed info please visit [this](https://en.wikipedia.org/wiki/Stable_Diffusion).

<img src="https://raw.githubusercontent.com/SlavyanDesu/sd-webui-colab/master/assets/image.png" width="100%">

This colab is based on [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab) and [TheLastBen/fast-stable-diffusion](https://github.com/TheLastBen/fast-stable-diffusion).

Github: [SlavyanDesu/sd-webui-colab](https://github.com/SlavyanDesu/sd-webui-colab)

In [ ]:
#@title ## Start Stable Diffusion
from IPython.display import clear_output
from google.colab import drive
import os
import time
import shutil

# Variables
model_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Stable-diffusion"
ext_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions"
vae_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/VAE"
lora_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Lora"
controlnet_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions/sd-webui-controlnet/models"
embd_path = "/content/gdrive/MyDrive/stable-diffusion-webui/embeddings"

#@markdown <small><font color=gray> **HINT**: Check `Use_Gdrive` to use your Google Drive storage.</small><br>
Use_Gdrive = False #@param {type:"boolean"}

if Use_Gdrive == True:
  drive.mount('/content/gdrive')

if not os.path.exists("/content/gdrive/MyDrive/"):
  print("Google drive not connected, using Colab storage instead...")
  !mkdir -p /content/gdrive/MyDrive/

# Cloning Repo
%cd /content/gdrive/MyDrive
!git clone --depth 1 -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git reset --hard
!rm webui.sh
!git pull
clear_output()

# Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

%pip install --upgrade fastapi==0.90.1
%pip install huggingface-hub==0.11.1
clear_output()

# Model Selection
Model_Name = "SD v1.5" #@param ["SD v1.5", "ChilloutMix", "Grapefruit v4.1", "Counterfeit v2.5", "Perfect World v2", "dalcefo_painting 3rd", "MIX-Pro-V3", "Meow Mix"]
#@markdown <small><font color=gray> **HINT**: Select model that you want to use.</small><br>
#@markdown Or
#@markdown <small><font color=gray> **HINT**: You can use any model by filling `Model_Link` to model URL and `Model_Filename` as model name.</small><br>
Model_Link = "" #@param {type:"string"}
Model_Filename = "" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}

if Model_Link != "":
  time.sleep(1)
  %cd $model_path
  if Model_Filename is None:
    Model_Filename = "model"
  if not safetensors:
    modelname = Model_Filename + ".ckpt"
  else:
    modelname = Model_Filename + ".safetensors"
  time.sleep(1)
  !wget $Model_Link -O "$model_path/$modelname"
  if os.path.exists(f"{model_path}/{modelname}"):
    print("Model downloaded! Using", modelname)
  else:
    print("Download failed! Please check the link...")
elif Model_Name == "SD v1.5":
  time.sleep(1)
  %cd $model_path
  !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O "$model_path/v1-5-pruned-emaonly.ckpt"
  if os.path.exists(f"{model_path}/v1-5-pruned-emaonly.ckpt"):
    print("Model downloaded! Using", Model_Name)
  else:
    print("Download failed! Please check the link...")
elif Model_Name == "ChilloutMix":
  time.sleep(1)
  %cd $model_path
  !wget https://civitai.com/api/download/models/11745 -O "$model_path/chilloutmix_NiPrunedFp32Fix.safetensors"
  if os.path.exists(f"{model_path}/chilloutmix_NiPrunedFp32Fix.safetensors"):
    print("Model downloaded! Using", Model_Name)
  else:
    print("Download failed! Please check the link...")
elif Model_Name == "Grapefruit v4.1":
  time.sleep(1)
  %cd $model_path
  !wget https://civitai.com/api/download/models/16619 -O "$model_path/grapefruitHentaiModel_grapefruitv41.safetensors"
  if os.path.exists(f"{model_path}/grapefruitHentaiModel_grapefruitv41.safetensors"):
    print("Model downloaded! Using", Model_Name)
  else:
    print("Download failed! Please check the link...")
elif Model_Name == "Counterfeit v2.5":
  time.sleep(1)
  %cd $model_path
  !wget https://civitai.com/api/download/models/7425 -O "$model_path/CounterfeitV25_25.safetensors"
  if os.path.exists(f"{model_path}/CounterfeitV25_25.safetensors"):
    print("Model downloaded! Using", Model_Name)
  else:
    print("Download failed! Please check the link...")
clear_output()


# Extension
!git clone https://github.com/SlavyanDesu/sd-webui-tunnels.git $ext_path/sd-webui-tunnels
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git $ext_path/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git $ext_path/ultimate-upscale-for-automatic1111
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git $ext_path/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser.git $ext_path/sd-civitai-browser
!git clone https://github.com/catppuccin/stable-diffusion-webui.git $ext_path/catppuccin-theme
!git clone https://github.com/Mikubill/sd-webui-controlnet.git $ext_path/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor.git $ext_path/openpose-editor
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon.git $ext_path/a1111-sd-webui-locon
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface.git $ext_path/stable-diffusion-webui-huggingface
!git clone https://github.com/kohya-ss/sd-webui-additional-networks.git $ext_path/sd-webui-additional-networks
clear_output()

# ControlNet
ControlNet = False #@param {type:"boolean"}
#@markdown <small><font color=gray> **HINT**: Check `ControlNet` to download ControlNet models.</small><br>
if ControlNet == True:
  time.sleep(1)
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -P $controlnet_path
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -P $controlnet_path
  clear_output()

# VAE
!wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -P $vae_path
!wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P $vae_path
!wget https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -P $vae_path
!wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P $vae_path
!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -P $vae_path

shutil.rmtree(embd_path)
!git clone https://huggingface.co/SlavyanDesu/embeddings
clear_output()

# Run Stable Diffusion
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
!COMMANDLINE_ARGS="--share --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@title ## Load existing Stable Diffusion
#@markdown Run this if you have Stable Diffusion installed before in your Google Drive.
# Update
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git reset --hard
!rm webui.sh
!git pull

# Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

%pip install --upgrade fastapi==0.90.1
%pip install huggingface-hub==0.11.1
clear_output()

# Run Stable Diffusion
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
!COMMANDLINE_ARGS="--share --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae" REQS_FILE="requirements.txt" python launch.py